# 1. import library

In [ ]:
# torch
import torch
import torch.nn as nn
import torchvision.datasets as dset # MNIST dataset 사용
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.optim as optim


# matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# 2. hyperarameter 설정

In [ ]:
# batchsize ,learning rate , num_epoch
batch_size = 16
learning_rate = 0.002
num_epoch = 200

# 3. model 정의

In [ ]:
class myCNN(nn.Module):
    def __init__(self):
        super(myCNN , self).__init__()
        # 1단게 : CNN layer
        self.cnn_layer = nn.Sequential(
            # convolution + relu --> 1 , 16 , 5(5x5 filter) . padding = 2
            nn.Conv2d(1 , 16 , 5 , padding=2), # 28x28x1 --> 28x28x16
            nn.ReLU(),

            # conv + relu --> 16 , 32 , 5(5x5 filter) , padding = 2
            nn.Conv2d(16 , 32 , 5 ,padding=2), # 28x28x16 --> 28x28x32
            nn.ReLU(),

            # pooling : 28x28 --> 14x14
            nn.MaxPool2d(2 , 2),

            # conv + relu --> 32 , 64 , 5 , padding = 2
            nn.Conv2d(32 , 64 , 5 , padding=2),
            nn.ReLU(),

            # pooling : 14x14 --> 7x7
            nn.MaxPool2d(2 ,2)
        ) # cnn_layer의 출력 : 7x7x64

        # 2단계 : FC layer (fully_connected)
        self.fc_layer = nn.Sequential(
            nn.Linear(64*7*7 , 100),
            nn.ReLU(),
            nn.Linear(100 , 10)
        )

    def forward (self , x):
        out = self.cnn_layer(x) # out : (bach_size) * 7x7x64 3d tensor
        out = out.view(batch_size , -1) # out을 펴야됨 out : bach_size x 7x7x64 --> 2d tensor
        out = self.fc_layer(out) # fc_layer의 input : 7x7x64x1 1d tensor
        # data를 batch_size 단위로 처리한다고 생각
        return out


# 4. data loading

In [ ]:
mnist_train = dset.MNIST("../" , train=True , transform=transforms.ToTensor(),
                         target_transform=None , download=True)
mnist_test = dset.MNIST("../", train=False , transform=transforms.ToTensor(),
                        target_transform=None , download=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:03<00:00, 2944593.94it/s]


Extracting ../MNIST/raw/train-images-idx3-ubyte.gz to ../MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 160155.71it/s]


Extracting ../MNIST/raw/train-labels-idx1-ubyte.gz to ../MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 1506562.76it/s]


Extracting ../MNIST/raw/t10k-images-idx3-ubyte.gz to ../MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 7601966.79it/s]

Extracting ../MNIST/raw/t10k-labels-idx1-ubyte.gz to ../MNIST/raw



# 5. data loader 설정

In [ ]:
train_loader = torch.utils.data.DataLoader( list(mnist_train)[:batch_size*100] , batch_size=batch_size,
                                           shuffle=True , num_workers=2 , drop_last=True)

test_loader = torch.utils.data.DataLoader( (mnist_test) , batch_size=batch_size,
                                          shuffle=False , num_workers=2 , drop_last=True)

# 6. optimizer 설정

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = myCNN().to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters() , lr=learning_rate)

# 7. accuracy 측정 함수

In [ ]:
def EstimateAccuracy (dloader , imodel):
    correct = 0
    total = 0

    for image,label in dloader:
        x = Variable(image , volatile=True).to(device) # backpropagation 안해서 volatitle = True
        y = Variable(label).to(device)

        y_hat = imodel.forward(x)
        _ , y_hat_index = torch.max(y_hat , 1)

        total += label.size(0)
        correct += (y_hat_index == y).sum().float()

        print("Accuracy : {}".format(100*correct/total))

        return 100*correct/total

# 8. 훈련

In [ ]:
loss_arr = []
accu_arr = []

for i in range(num_epoch):
    for image , label in train_loader:
        x = Variable(image).to(device)
        y = Variable(label).to(device)

        optimizer.zero_grad()
        y_hat = model.forward(x)
        loss = loss_func(y_hat , y)
        loss.backward()
        optimizer.step()
    if i%10 == 0:
        print(i , loss)
        accu = EstimateAccuracy(test_loader , model)
        loss_arr.append(loss)
        accu_arr.append(accu)

# 9. 결과 출력